In [1]:
!pip install geopy

In [13]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from geopy.geocoders import Nominatim
import re
import json
import requests

### 전처리
data = pd.read_csv('대전광역시_온통대전 신청 가맹점 목록_20211130.csv',encoding='cp949')
data.columns
data.drop(columns=['순번'],inplace=True)
data.dropna(axis=0, subset=['가맹점'], inplace=True)
data.isna().sum()
data['주소']=data['주소'].str.split(',').str.get(0) 
data['주소']=data['주소'].str.split('(').str.get(0) 
donggu = data[data['구']=='동구']
donggu.reset_index(drop=True, inplace=True)
donggu

,구,가맹점,주소,동
0,동구,음주측정,대전광역시 동구 백룡로 19,자양동
1,동구,(ACE)에이스 전자저울,대전광역시 동구 동서대로1730번길 28,가양동
2,동구,(동진보석),대전광역시 동구 가양1동 318 홍명상가 C-33호,가양동
3,동구,(본만쥬)김양경김밥.초봉김밥,대전광역시 동구 한남로 5,홍도동
4,동구,(사)대전광역시관광협회,대전광역시 동구 중앙로 187-1,중동
...,...,...,...,...
5513,동구,힐링스토리,대전광역시 동구 동대전로 225,자양동
5514,동구,힐링앤미,대전광역시 동구 용운로 80,용운동
5515,동구,힐링에스테틱,대전광역시 동구 흥룡로 60,가양동
5516,동구,힐링타이마사지,대전광역시 동구 은어송로 34,가오동


In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import re
from geopy.geocoders import Nominatim
import json
import requests


REST_API_KEY = "24e7619a54b8e426d7f7e06a44973320"



data = pd.read_csv("대전광역시_온통대전 신청 가맹점 목록_20211130.csv",encoding='EUC-KR')

data.dropna(axis=0, subset=['가맹점'], inplace=True)

data['주소']=data['주소'].str.split(',').str.get(0)
data['주소']=data['주소'].str.split('(').str.get(0)

data.reset_index(drop=True, inplace=True)

data["경도"] = ""
data["위도"] = ""


def find_coord_kakao(address):
    def find_coord_geo(address):
        try:
            geolocoder = Nominatim(user_agent="South Korea", timeout=None)
            geo = geolocoder.geocode(address)

            return geo.latitude, geo.longitude
        except:
            return 0, 0
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json?query={address}".format(address = address)
        headers = {"Authorization": f"KakaoAK {REST_API_KEY}"}
        result = json.loads(str(requests.get(url, headers=headers).text))
        coord = result["documents"][0]["address"]
        return float(coord["x"]), float(coord["y"])
    except:
        return find_coord_geo(address)

for i in data.index:
    data.loc[i, "경도"] = find_coord_kakao(data.loc[i, "주소"])[0]
    data.loc[i, "위도"] = find_coord_kakao(data.loc[i, "주소"])[1]
    print(i)